Multi-resolution registration example

# Working with big data

## Strategy 1 - downsample

Registration algorithms can be expensive, but will be much faster if working on lower resolution images. In this notebook, we will 

1. run a registration algorithm on the downsampled images we produced in `resample_images.ipynb`
2. apply the transformation that results on the original, high-resolution images.

Two facts make this possible.

* the original and downsampled images are in the same **physical** coordinate system.
* the transformation produced by teh registration works on **physical** (not pixel) coordinates.

First, let's import libraries and make some useful functions:

In [1]:
import os
from itertools import chain
import numpy as np

import SimpleITK as sitk

def run_elastix(fixed_image, moving_image, params):
    """
    Runs elastix with the given fixed and moving images,
    and elastix parameters.
    
    Returns the ElastixImageFilter, from which the results
    can be obtained.
    """
    elastixImageFilter = sitk.ElastixImageFilter()
    elastixImageFilter.SetFixedImage(fixed_image)
    elastixImageFilter.SetMovingImage(moving_image)
    elastixImageFilter.SetParameterMap(params)
    elastixImageFilter.Execute()
    return elastixImageFilter

def print_parameters(elastix_parameters):
    for k,v in elastix_parameters.iteritems():
        print(f'{k} \n\t {v}')

Next, we'll load the downsampled images.

In [5]:
# load fixed image
fixed_image_path='../sampleImages/jrc18_down.nrrd'
fixed_image = sitk.ReadImage(fixed_image_path)

# load moving image
moving_image_path = '../sampleImages/jrc10_down.nrrd'
moving_image = sitk.ReadImage(moving_image_path)

In [14]:
%%time

# run a simple registration
affine_params = sitk.ReadParameterFile('../elastixParameters/DefaultSmoother_Affine.txt')
elastixImageFilter = run_elastix(fixed_image, moving_image, affine_params ) 

# write the output to a file
sitk.WriteImage(elastixImageFilter.GetResultImage(), '../sampleImages/affine_result_img.nrrd')


Time spent on saving the results, applying the final transform etc.: 254 ms.
ELASTIX version: 5.000
Command line options from ElastixBase:
-fMask    unspecified, so no fixed mask used
-mMask    unspecified, so no moving mask used
-out      ./
-threads  unspecified, so all available threads are used
Command line options from TransformBase:
-t0       unspecified, so no initial transform used

Reading images...
Reading images took 0 ms.

  A default pyramid schedule is used.
  A default pyramid schedule is used.
  The default value "false" is used instead.
  The default value "GeometricalCenter" is used instead.
Transform parameters are initialized as: [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]
InitializeTransform took 0.00s
  The default value "false" is used instead.
Scales for transform parameters are: [100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 1, 1, 1]
Initialization of all components (before registration) took: 0 ms.
Preparation of the image pyramids took: 69 

383	-0.906145	3.722737	64.021207	0.000649	0.3
384	-0.891633	3.701725	64.075667	0.001012	0.3
385	-0.898402	3.684409	64.120613	0.001056	0.3
386	-0.903031	3.666160	64.168054	0.000964	0.3
387	-0.892827	3.644074	64.225562	0.001567	0.3
388	-0.887558	3.621865	64.283492	0.000694	0.3
389	-0.892930	3.601954	64.335519	0.001009	0.3
390	-0.887280	3.580618	64.391363	0.000937	0.3
391	-0.891095	3.561869	64.440514	0.001240	0.3
392	-0.899978	3.539489	64.499284	0.001052	0.3
393	-0.897747	3.517080	64.558237	0.000974	0.3
394	-0.887818	3.500502	64.601920	0.000891	0.3
395	-0.892352	3.478414	64.660212	0.000791	0.3
396	-0.900379	3.473989	64.671905	0.001437	0.3
397	-0.898062	3.451668	64.730942	0.000698	0.3
398	-0.897067	3.466011	64.692993	0.000772	0.3
399	-0.895966	3.446754	64.743953	0.000808	0.3
400	-0.899889	3.425559	64.800132	0.001134	0.3
401	-0.889288	3.493654	64.619981	0.000773	0.3
402	-0.897548	3.490326	64.628763	0.000744	0.3
403	-0.897882	3.468652	64.686009	0.000780	0.3
404	-0.896427	3.459768	64.709505	0

  Sampling the gradients took 0.033697s
Automatic parameter estimation took 0.03s
1:ItNr	2:Metric	3a:Time	3b:StepSize	4:||Gradient||	Time[ms]
0	-0.843199	0.000000	161.467273	0.008619	35.4
1	-0.862016	0.000000	161.467273	0.006151	0.4
2	-0.884026	0.000000	161.467273	0.005134	0.4
3	-0.883277	0.000000	161.467273	0.005146	0.4
4	-0.879648	0.000000	161.467273	0.004531	0.4
5	-0.862369	0.000000	161.467273	0.004925	0.4
6	-0.899377	0.000000	161.467273	0.003158	0.3
7	-0.885188	0.000000	161.467273	0.003233	0.3
8	-0.892235	0.000000	161.467273	0.002482	0.3
9	-0.888258	0.000000	161.467273	0.002048	0.3
10	-0.889016	0.000000	161.467273	0.002152	0.3
11	-0.876269	0.999994	154.127893	0.002012	0.4
12	-0.875899	0.961853	154.395569	0.002377	0.4
13	-0.900697	0.923710	154.664187	0.001497	0.4
14	-0.885355	0.885567	154.933743	0.001446	0.4
15	-0.893035	0.847423	155.204239	0.002219	0.4
16	-0.888938	0.809339	155.475264	0.001645	0.4
17	-0.886425	0.771196	155.747656	0.002422	0.4
18	-0.881880	0.734059	156.013783	0.0017

617	-0.895146	118.793133	24.255932	0.001143	0.3
618	-0.915565	118.755533	24.262458	0.000862	0.3
619	-0.910306	118.717459	24.269070	0.001593	0.3
620	-0.900220	118.680303	24.275525	0.001351	0.3
621	-0.910432	119.592010	24.118104	0.000873	0.3
622	-0.897435	119.554353	24.124566	0.001108	0.3
623	-0.917856	119.516257	24.131106	0.000872	0.3
624	-0.908886	119.480573	24.137236	0.001646	0.3
625	-0.901128	120.437048	23.974007	0.001026	0.3
626	-0.900338	120.398904	23.980474	0.001106	0.3
627	-0.902549	120.360761	23.986944	0.002352	0.3
628	-0.911989	121.360761	23.818450	0.000837	0.3
629	-0.916784	121.322618	23.824834	0.000974	0.3
630	-0.896055	121.284508	23.831215	0.000817	0.3
631	-0.906061	121.251789	23.836697	0.001512	0.3
632	-0.910577	121.213646	23.843090	0.000826	0.3
633	-0.905797	122.211173	23.677012	0.000975	0.3
634	-0.906193	122.694535	23.597367	0.000696	0.3
635	-0.911520	122.656436	23.603626	0.001062	0.3
636	-0.906552	122.794306	23.580995	0.001277	0.3
637	-0.911364	123.668801	23.438452	0.000

87	-0.876039	29.515743	82.485889	0.002874	0.4
88	-0.904565	29.334310	82.783215	0.001505	0.3
89	-0.870801	30.215164	81.359421	0.003275	0.3
90	-0.877089	31.215017	79.801487	0.002193	0.3
91	-0.883858	31.033916	80.079233	0.003031	0.3
92	-0.885919	30.852809	80.358926	0.001999	0.3
93	-0.899812	30.671438	80.640992	0.001492	0.4
94	-0.890464	31.665708	79.118580	0.001698	0.4
95	-0.886183	31.484378	79.391929	0.001951	0.4
96	-0.881460	32.184900	78.346223	0.002015	0.3
97	-0.889954	32.081969	78.498144	0.002248	0.4
98	-0.884167	31.900536	78.767369	0.001670	0.4
99	-0.889622	31.827005	78.877007	0.001270	0.3
100	-0.871542	31.666939	79.116730	0.001747	0.4
101	-0.885973	31.651935	79.139276	0.001807	0.4
102	-0.888591	31.486300	79.389023	0.001442	0.3
103	-0.887791	31.305375	79.663630	0.002179	0.4
104	-0.858025	32.297735	78.180358	0.002404	0.4
105	-0.889639	33.297726	76.740526	0.001509	0.4
106	-0.898750	34.154446	75.548507	0.001622	0.4
107	-0.881591	35.141454	74.220308	0.002274	0.4
108	-0.890319	36.045240	73

637	-0.886046	208.439182	18.160961	0.002941	0.4
638	-0.892564	208.347707	18.168204	0.001068	0.3
639	-0.892429	208.277350	18.173780	0.001038	0.4
640	-0.882697	208.177173	18.181724	0.002456	0.4
641	-0.902892	208.000120	18.195781	0.001472	0.4
642	-0.878765	208.778242	18.134163	0.001990	0.4
643	-0.882183	208.617448	18.146861	0.002387	0.5
644	-0.887199	208.436036	18.161210	0.002158	0.6
645	-0.885624	208.254605	18.175583	0.001566	0.5
646	-0.880701	208.078959	18.189519	0.004118	0.4
647	-0.879746	207.897526	18.203937	0.002312	0.4
648	-0.879137	208.324591	18.170036	0.001778	0.4
649	-0.882911	208.208692	18.179223	0.002192	0.4
650	-0.881770	208.042769	18.192393	0.002999	0.4
651	-0.882090	207.868599	18.206237	0.002845	0.4
652	-0.884670	208.868598	18.127035	0.001176	0.4
653	-0.871928	208.698322	18.140472	0.003096	0.4
654	-0.882192	208.517063	18.154798	0.001555	0.4
655	-0.887040	208.383213	18.165392	0.001034	0.4
656	-0.898838	209.024283	18.114766	0.002231	0.5
657	-0.891371	209.771843	18.056085	0.001

  Sampling the gradients took 0.024177s
Automatic parameter estimation took 0.03s
1:ItNr	2:Metric	3a:Time	3b:StepSize	4:||Gradient||	Time[ms]
0	-0.850335	0.000000	147.691705	0.002901	65.6
1	-0.866997	0.000000	147.691705	0.005328	0.6
2	-0.852906	0.999999	140.978450	0.005476	0.6
3	-0.879219	1.603286	137.215706	0.007720	0.4
4	-0.876429	1.163738	139.936945	0.003420	0.4
5	-0.850821	2.163738	133.895738	0.003155	0.4
6	-0.845799	3.162472	128.361280	0.003367	0.5
7	-0.875625	2.791277	130.363992	0.005059	0.4
8	-0.867834	2.351400	132.819694	0.003293	0.4
9	-0.835552	3.334830	127.452124	0.005675	0.4
10	-0.847368	2.894232	129.802284	0.005929	0.4
11	-0.845747	3.894232	124.588132	0.002943	0.4
12	-0.844921	4.894201	119.776850	0.003273	0.4
13	-0.870516	4.454960	121.843670	0.002625	0.4
14	-0.858744	5.434094	117.330513	0.005852	0.4
15	-0.870057	4.993514	119.319219	0.003035	0.5
16	-0.864708	4.552914	121.376597	0.005734	0.6
17	-0.857359	5.552910	116.805495	0.001507	0.6
18	-0.865732	6.544994	112.598528	0.0022

468	-0.857443	119.970019	22.001315	0.003144	0.4
469	-0.860243	120.969457	21.846430	0.003351	0.5
470	-0.841992	120.529097	21.914404	0.005447	0.4
471	-0.872614	121.529097	21.760650	0.003564	0.4
472	-0.859408	121.128370	21.822004	0.001580	0.4
473	-0.860787	122.122511	21.670426	0.002646	0.4
474	-0.850808	121.789673	21.720939	0.004123	0.4
475	-0.857328	121.349091	21.788167	0.006248	0.5
476	-0.862966	122.349091	21.636174	0.004140	0.4
477	-0.864120	123.348921	21.486311	0.004479	0.4
478	-0.829215	123.237607	21.502893	0.004427	0.4
479	-0.841919	123.516541	21.461390	0.005029	0.4
480	-0.873487	123.096954	21.523882	0.003524	0.4
481	-0.844601	124.062701	21.380588	0.004903	0.4
482	-0.847885	123.622103	21.445725	0.002853	0.4
483	-0.856540	124.584239	21.303994	0.004096	0.4
484	-0.868457	124.343271	21.339315	0.002726	0.4
485	-0.863894	125.331592	21.195189	0.003221	0.4
486	-0.866602	124.936997	21.252498	0.003803	0.4
487	-0.873752	124.501112	21.316166	0.003538	0.4
488	-0.840830	124.060674	21.380887	0.003

940	-0.866020	235.611642	12.086458	0.005474	1.1
941	-0.865725	235.171042	12.107246	0.004521	0.5
942	-0.867157	236.171042	12.060167	0.003318	0.4
943	-0.867213	237.166167	12.013680	0.003321	0.4
944	-0.848779	236.725567	12.034219	0.002560	0.5
945	-0.857986	236.303646	12.053952	0.002605	0.4
946	-0.877551	237.278818	12.008440	0.004552	0.5
947	-0.852246	238.278797	11.962127	0.003092	0.5
948	-0.857178	237.838197	11.982489	0.002893	0.6
949	-0.858095	237.810240	11.983783	0.002292	0.6
950	-0.868417	238.770997	11.939461	0.003674	0.4
951	-0.865654	238.425198	11.955376	0.001426	0.4
952	-0.873948	238.141436	11.968467	0.004565	0.5
953	-0.871663	238.862229	11.935270	0.003063	0.5
954	-0.845579	238.421636	11.955540	0.004962	0.6
955	-0.878660	237.981037	11.975880	0.003146	0.6
956	-0.860151	238.981037	11.929816	0.003085	0.6
957	-0.865310	239.016047	11.928209	0.002276	0.6
958	-0.854504	239.011724	11.928408	0.001649	0.5
959	-0.859237	238.923831	11.932441	0.003163	0.6
960	-0.883666	239.890600	11.888224	0.005

In [16]:
# print the output
transform_parameters = elastixImageFilter.GetTransformParameterMap()
print_parameters(transform_parameters[0])

CenterOfRotationPoint 
	 ('313.12', '145.16', '90.44')
CompressResultImage 
	 ('false',)
DefaultPixelValue 
	 ('0',)
Direction 
	 ('1', '0', '0', '0', '1', '0', '0', '0', '1')
FinalBSplineInterpolationOrder 
	 ('3',)
FixedImageDimension 
	 ('3',)
FixedInternalImagePixelType 
	 ('float',)
HowToCombineTransforms 
	 ('Compose',)
Index 
	 ('0', '0', '0')
InitialTransformParametersFileName 
	 ('NoInitialTransform',)
MovingImageDimension 
	 ('3',)
MovingInternalImagePixelType 
	 ('float',)
NumberOfParameters 
	 ('12',)
Origin 
	 ('0', '0', '0')
ResampleInterpolator 
	 ('FinalBSplineInterpolator',)
Resampler 
	 ('DefaultResampler',)
ResultImageFormat 
	 ('nrrd',)
ResultImagePixelType 
	 ('float',)
Size 
	 ('413', '192', '120')
Spacing 
	 ('1.52', '1.52', '1.52')
Transform 
	 ('AffineTransform',)
TransformParameters 
	 ('1.00567', '0.0255456', '0.0268564', '-0.0241198', '1.15818', '-0.078441', '-0.00286924', '0.0591024', '0.798849', '3.52426', '10.5495', '-27.9369')
UseDirectionCosines 
	 ('tr

This output contains lots of information, but notice three things in particular:

1. The `Size` field tells elastix what the pixel size of the output image 
2. The `Spacing` field tells elastix the resolution of the 
3. The `TransformParameters` store the affine transformation that will be applied to the image.

Next, we'll apply this transformation to the high resolution image, so the first step is to load it.

In [21]:
moving_image_hi_path = '../sampleImages/JFRCtemplate2010.nrrd'
moving_image_hi = sitk.ReadImage(moving_image_hi_path)
print( moving_image_hi.GetSpacing() )

(0.622088, 0.622088, 0.622088)


Finally, we'll apply the transformation to this higher resolution image.

Elastix's default behavior is to transform images to match the size and resolution of the fixed image. In this case, our downsampled fixed image has resolution `1.52 x 1.52 x 1.52`, and its size is `413 x 192 x 120`, so that is the size and resolution of the output we would get.

What we want is an output image matching the original high-resolution fixed image, which has resolution `0.38 x 0.38 x 0.38` and size `1652 x 768 x 479`. We need to tell elastix that this is what we want, so we set the appropriate values of the transform parameters, like so:

In [18]:
transform_parameters_highres = elastixImageFilter.GetTransformParameterMap()
transform_parameters_highres[0]['Size'] = ['1652', '768', '479']
transform_parameters_highres[0]['Spacing'] = ['0.38', '0.38', '0.38']

and now we can use those edited parameters to apply the transformation.

In [19]:
# "Transformix" is the filter that comes with elastix that transforms images  
transformixImageFilter = sitk.TransformixImageFilter()

# set the transformation and moving image, and run 
transformixImageFilter.SetTransformParameterMap(transform_parameters_highres)
transformixImageFilter.SetMovingImage(moving_image_hi)

# run it
transformixImageFilter.Execute()

# write the resulting image
sitk.WriteImage(transformixImageFilter.GetResultImage(), '../sampleImages/affine_result_hires_img.nrrd')

Time spent on saving the results, applying the final transform etc.: 246 ms.
ELASTIX version: 5.000
Command line options from ElastixBase:
-out      ./
-threads  unspecified, so all available threads are used
-def      unspecified, so no input points transformed
-jac      unspecified, so no det(dT/dx) computed
-jacmat   unspecified, so no dT/dx computed

Reading input image ...
  Reading input image took 0.000002 s
Calling all ReadFromFile()'s ...
  The default value "false" is used instead.
  Calling all ReadFromFile()'s took 0.000062 s
Transforming points ...
  The command-line option "-def" is not used, so no points are transformed
  Transforming points done, it took 0.00s
Compute determinant of spatial Jacobian ...
  The command-line option "-jac" is not used, so no det(dT/dx) computed.
  Computing determinant of spatial Jacobian done, it took 0.00s
Compute spatial Jacobian (full matrix) ...
  The command-line option "-jacmat" is not used, so no dT/dx computed.
  Computing spatial 

 ## Check the results
 
 Use Fiji to open the resulting image, located in `sampleImages/affine_result_hires_img.nrrd`. Confirm that:
 
 * It is the expected size and resolution
 * It is well registered to the high-resolution target image

 ## Bonus (optional)
 
Run transformix again using the higher resolution moving image, but this time use transform parameters directly from elastix `transform_parameters = elastixImageFilter.GetTransformParameterMap()`.  (i.e., don't set the output size and resolution).  Confirm that the resulting image:

* Is the expected, smaller size and resolution
* Is still well registered to the fixed image.


## Advanced (optional)

Run transformix again with the high resolution image, but this time create an image 
at resolution `[0.5, 0.5, 0.75]`.

1. determine a reasonable pixel size for the resulting image.
2. run `transform_parameters = elastixImageFilter.GetTransformParameterMap()`
2. set the values for the `transform_parameters` appropriately.
3. run transformix
4. verify that the result is as expected
